Problem statement

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!



To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

Help save them and change history!

Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load files

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read files

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse train

In [ ]:
train.isnull().sum()

In [ ]:
train.drop(['Cabin', 'Name'], axis = 1, inplace=True)
test.drop(['Cabin', 'Name'], axis = 1, inplace=True)
train

In [ ]:
test

In [ ]:
train1 = train.dropna()
train1

Analyse target

In [ ]:
train1['Transported'].replace({False: 0, True: 1},inplace=True)
train1['Transported']

In [ ]:
sns.displot(train1['Transported'])

In [ ]:
trans_count = train1['Transported'].value_counts()
trans_count

In [ ]:
trans_percent = trans_count / len(train1)
trans_percent

In [ ]:
plt.figure(figsize=(25, 7))
ax = plt.subplot()
ax.scatter(train1[train1['Transported'] == 1]['Age'], train1[train['Transported'] == 1]['FoodCourt'], c='green', s=train1[train1['Transported'] == 1]['VRDeck'])
ax.scatter(train1[train1['Transported'] == 0]['Age'], train1[train['Transported'] == 0]['FoodCourt'], c='red', s=train1[train1['Transported'] == 0]['VRDeck']);


Drop target

In [ ]:
target = train1['Transported']

train1.drop(['Transported'],axis=1, inplace=True)
train1

Combine train and test

In [ ]:
combi = train1.append(test)
combi

Analyse combi

In [ ]:
combi.info()

In [ ]:
combi.describe()

Check for null values

In [ ]:
combi.isnull().sum()

Impute null values

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(random_state=42)

date = pd.Timestamp('2200-01-01')

for col in combi:
    if combi[col].dtype=="object":
        combi[col].fillna("not listed", inplace=True)
    if combi[col].dtype=="int":
        #X[col].fillna(X[col].mode()[0], inplace=True)
        combi[col].fillna(combi[col].mean(), inplace=True)
        #combi[col] = combi[col].astype.int()
    if combi[col].dtype=='float':
       #X[col].fillna(X[col].mean(), inplace=True)
       combi[col] = imp.fit_transform(combi[col].values.reshape(-1,1))
    if combi[col].dtype=="datetime64[ns]":
        combi[col].fillna(date, inplace=True)
combi

In [ ]:
combi.isnull().sum()

Analyse features

Home Planet

In [ ]:
sns.displot(combi['HomePlanet'])

In [ ]:
home_count = combi['HomePlanet'].value_counts()
home_count

In [ ]:
home_percent = home_count / len(combi)
home_percent

In [ ]:
mylabels = ["Earth", "Europa", "Mars", "not listed"]
plt.pie(home_percent, labels=mylabels)
plt.show() 

In [ ]:
combi['HomePlanet'].replace({"Earth": 1, "Europa": 2, "Mars": 3, "not listed": 4},inplace=True)
combi['HomePlanet']

Cryo Sleep

In [ ]:
combi['CryoSleep'][combi['CryoSleep'] == 'not listed'] = False

In [ ]:
print(combi.iloc[6674])

In [ ]:
combi['CryoSleep'].replace({False: 0, True: 1})

In [ ]:
sns.distplot(combi['CryoSleep'])

In [ ]:
sleep_count = combi['CryoSleep'].value_counts()
sleep_count

In [ ]:
sleep_percent = sleep_count / len(combi)
sleep_percent

In [ ]:
combi['CryoSleep'] = combi['CryoSleep'].astype(int)
combi['CryoSleep']

Destination

In [ ]:
sns.displot(combi['Destination'])

In [ ]:
dest_count = combi['Destination'].value_counts()
dest_count

In [ ]:
dest_percent = dest_count / len(combi)
dest_percent

In [ ]:
mylabels = ["TRAPPIST-1e", "55 Cancri e", "PSO J318.5-22", "not listed"]
plt.pie(dest_percent, labels=mylabels)
plt.show() 

In [ ]:
combi['Destination'].replace({"TRAPPIST-1e": 1, "55 Cancri e": 2, "PSO J318.5-22": 3, "not listed": 4},inplace=True)
combi['Destination']

Age

In [ ]:
combi['Age_group'] = pd.cut(x=combi['Age'], bins=[-1, 18, 40, 65, 100], labels=['child', 'young adult', 'middle age', 'pensioner'])
combi['Age_group']

In [ ]:
sns.displot(combi['Age_group'])

In [ ]:
age_count = combi['Age_group'].value_counts()
age_count

In [ ]:
age_percent = age_count / len(combi)
age_percent

In [ ]:
mylabels = ["young adult", "child", "middle age", "pensioner"]
plt.pie(age_percent, labels=mylabels)
plt.show() 

In [ ]:
combi['Age_group'].replace({"young adult": 1, "child": 2, "middle age": 3, "pensioner": 4},inplace=True)
combi['Age_group']

In [ ]:
combi['Age_group'] = combi['Age_group'].astype(int)

VIP

In [ ]:
combi['VIP'][combi['VIP'] == 'not listed'] = False

In [ ]:
combi['VIP'].replace({False: 0, True: 1})

In [ ]:
sns.distplot(combi['VIP'])

In [ ]:
vip_count = combi['VIP'].value_counts()
vip_count

In [ ]:
vip_percent = vip_count / len(combi)
vip_percent

In [ ]:
combi['VIP'] = combi['VIP'].astype(int)
combi['VIP']

Room Service

In [ ]:
sns.violinplot(combi['RoomService'])

In [ ]:
rm_service_high = combi['RoomService'].max()
print(rm_service_high)

In [ ]:
combi['Room_Service_group'] = pd.cut(x=combi['RoomService'], bins=[-1, 2000, 8000, 15000], labels=['low', 'med', 'high'])
combi['Room_Service_group']


In [ ]:
sns.displot(combi['Room_Service_group'])

In [ ]:
rm_service_count = combi['Room_Service_group'].value_counts()
rm_service_count

In [ ]:
rm_service_percent = rm_service_count / len(combi)
rm_service_percent

In [ ]:
mylabels = ["low", "med", "high"]
plt.pie(rm_service_percent, labels=mylabels)
plt.show()

In [ ]:
combi['Room_Service_group'].replace({"low": 1, "med": 2, "high": 3},inplace=True)
combi['Room_Service_group']

Food Court

In [ ]:
sns.violinplot(combi['FoodCourt'])

In [ ]:
food_high = combi['FoodCourt'].max()
print(food_high)

In [ ]:
combi['Food_Court_group'] = pd.cut(x=combi['FoodCourt'], bins=[-1, 5000, 20000, 30000], labels=['low', 'med', 'high'])
combi['Food_Court_group']

In [ ]:
sns.displot(combi['Food_Court_group'])

In [ ]:
fd_court_count = combi['Food_Court_group'].value_counts()
fd_court_count

In [ ]:
fd_court_percent = fd_court_count / len(combi)
fd_court_percent

In [ ]:
mylabels = ["low", "med", "high"]
plt.pie(fd_court_percent, labels=mylabels)
plt.show()

In [ ]:
combi['Food_Court_group'].replace({"low": 1, "med": 2, "high": 3},inplace=True)
combi['Food_Court_group']

Shopping Mall

In [ ]:
sns.violinplot(combi['ShoppingMall'])

In [ ]:
shop_high = combi['ShoppingMall'].max()
print(shop_high)

In [ ]:
combi['Shopping_group'] = pd.cut(x=combi['ShoppingMall'], bins=[-1, 2000, 15000, 24000], labels=['low', 'med', 'high'])
combi['Shopping_group']

In [ ]:
sns.displot(combi['Shopping_group'])

In [ ]:
shopping_count = combi['Shopping_group'].value_counts()
shopping_count

In [ ]:
shopping_percent = shopping_count / len(combi)
shopping_percent

In [ ]:
mylabels = ["low", "med", "high"]
plt.pie(shopping_percent, labels=mylabels)
plt.show()

In [ ]:
combi['Shopping_group'].replace({"low": 1, "med": 2, "high": 3},inplace=True)
combi['Shopping_group']

Spa

In [ ]:
sns.violinplot(combi['Spa'])

In [ ]:
spa_high = combi['Spa'].max()
print(spa_high)

In [ ]:
combi['Spa_group'] = pd.cut(x=combi['Spa'], bins=[-1, 5000, 15000, 23000], labels=['low', 'med', 'high'])
combi['Spa_group']

In [ ]:
sns.displot(combi['Spa_group'])

In [ ]:
spa_count = combi['Spa_group'].value_counts()
spa_count

In [ ]:
spa_percent = spa_count / len(combi)
spa_percent

In [ ]:
mylabels = ["low", "med", "high"]
plt.pie(spa_percent, labels=mylabels)
plt.show()

In [ ]:
combi['Spa_group'].replace({"low": 1, "med": 2, "high": 3},inplace=True)
combi['Spa_group']

VR Deck

In [ ]:
sns.violinplot(combi['VRDeck'])

In [ ]:
vr_high = combi['VRDeck'].max()
print(vr_high)

In [ ]:
combi['VR_group'] = pd.cut(x=combi['VRDeck'], bins=[-1, 5000, 15000, 23000], labels=['low', 'med', 'high'])
combi['VR_group']

In [ ]:
sns.displot(combi['VR_group'])

In [ ]:
vr_count = combi['VR_group'].value_counts()
vr_count

In [ ]:
vr_percent = vr_count / len(combi)
vr_percent

In [ ]:
mylabels = ["low", "med", "high"]
plt.pie(vr_percent, labels=mylabels)
plt.show()

In [ ]:
combi['VR_group'].replace({"low": 1, "med": 2, "high": 3},inplace=True)
combi['VR_group']

Assign features

In [ ]:
combi.isnull().sum()

In [ ]:
combi.info()

Define X and y

In [ ]:
features = ["HomePlanet", "CryoSleep", "Destination", "Age_group", "Room_Service_group", "Food_Court_group", "Shopping_group", "Spa_group", "VR_group"]

y = target
X = combi[features][: len(train1)]
X_test = combi[features][len(train1) :]

Heatmap

In [ ]:
cmap = combi[features].corr()
sns.heatmap(cmap)


Split dataset for training and validating

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=1, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

Select model

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, C=10).fit(X_train, y_train)
print(model.score(X_train, y_train))

Predict validation

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_val, y_pred))

Predict on X_test

In [ ]:
predictions = model.predict(X_test)
predictions = predictions.astype(str)
predictions = np.char.replace(predictions, '0', 'False')
predictions = np.char.replace(predictions, '1', 'True')
predictions

Prepare and submit

In [ ]:
submission['Transported'] = predictions
submission.to_csv('submission.csv', index=False)
my_submission = pd.read_csv("submission.csv")
my_submission